<a href="https://colab.research.google.com/github/KAMEHAMEHA-05/Sky-Beat/blob/main/Sky_Beat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cv2
!pip install numpy
!pip install random
!pip install time
!pip install statistics
!pip install keras.models
!pip install utils
!pip install firestore
!pip install utilx
!pip install oauth2
!pip install numpy
!pip install SpeechRecognition
!pip install textblob
!pip install wavio
!pip install ffmpeg-python
!pip install spotify==0.4.8
!pip install spotipy==2.4.4
!pip install python-oauth2
!pip install delegator
!pip install spotify_local

ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2
ERROR: Could not find a version that satisfies the requirement random (from versions: none)
ERROR: No matching distribution found for random
ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time
  Preparing metadata (setup.py) ... done
  Created wheel for statistics: filename=statistics-1.0.3.5-py3-none-any.whl size=7436 sha256=8e33827af430632c0cbae1724be10a2fd5e13a6eeaa145edbeacd7a4ac198b4a
  Stored in directory: /root/.cache/pip/wheels/59/bd/aa/41ad56fbb723d36e457b5be74feb992cdac2855a6999ba34a1
Successfully built statistics
  Preparing metadata (setup.py) ... done
  Created wheel for utils: filename=utils-1.0.2-py2.py3-none-any.whl size=13905 sha256=ddcb04b63fd97044397746d0442d07d871ed93d2f78aafcf709d337d9e61f406
  Stored in directory: /root/.cache/pip/wheels/b8/39/f5/9

In [ ]:
from google.colab import files
src = list(files.upload().values())

Saving __init__.py to __init__.py
Saving __init__.pyc to __init__.pyc
Saving data_augmentation.py to data_augmentation.py
Saving datasets.py to datasets.py
Saving datasets.pyc to datasets.pyc
Saving emotion_model.hdf5 to emotion_model.hdf5
Saving grad_cam.py to grad_cam.py
Saving haarcascade_frontalface_default.xml to haarcascade_frontalface_default.xml
Saving inference.py to inference.py
Saving inference.pyc to inference.pyc
Saving preprocessor.py to preprocessor.py
Saving preprocessor.pyc to preprocessor.pyc
Saving visualizer.py to visualizer.py


In [ ]:
import cv2
import numpy as np
from keras.models import load_model
from statistics import mode
import os
os.chdir("/content/")
from datasets import *
import inference
from inference import draw_text
from inference import draw_bounding_box
from inference import apply_offsets
from inference import load_detection_model
import preprocessor
import random
import time
import spotipy #as sp
import spotify
from __future__ import print_function
import time
import sys
from python_utils import*
import os
from spotify_local import SpotifyLocal
import spotipy.util as utilx
import oauth2

from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
import math, random
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
import speech_recognition
from textblob import TextBlob
import wavio
import numpy as np
from __future__ import print_function
import time
import spotify
import sys
import spotipy
from python_utils import*
import os
from spotify_local import SpotifyLocal
import spotipy.util as utilx
import oauth2
import random
from google.colab import drive

from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from IPython.display import Image
import cv2
import numpy as np

#Capturing
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  print(filename)
  return filename

filename = take_photo()

#Facial
i=0
USE_WEBCAM = True
Emotion_score_array=[0.5]*30

emotion_model_path = 'emotion_model.hdf5'
emotion_labels = get_labels('fer2013')


frame_window = 10
emotion_offsets = (20, 40)

# loading models
face_cascade = cv2.CascadeClassifier('/content/haarcascade_frontalface_default.xml')
emotion_classifier = load_model('/content/emotion_model.hdf5')


emotion_target_size = emotion_classifier.input_shape[1:3]


emotion_window = []

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""
while 1<10 :



    image = cv2.imread("/content/photo.jpg")
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5,
    minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)

    for face_coordinates in faces:

        x1, x2, y1, y2 = apply_offsets(face_coordinates, emotion_offsets)
        gray_face = gray_image[y1:y2, x1:x2]

        try:
            gray_face = cv2.resize(gray_face, (emotion_target_size))

        except:
            continue

            gray_face = preprocess_input(gray_face, True)
            gray_face = np.expand_dims(gray_face, 0)
            gray_face = np.expand_dims(gray_face, -1)
            emotion_prediction = emotion_classifier.predict(gray_face)
            emotion_probability = np.max(emotion_prediction)
            emotion_label_arg = np.argmax(emotion_prediction)
            emotion_text = emotion_labels[emotion_label_arg]
            emotion_window.append(emotion_text)
            print("g")

        # if len(emotion_window) > frame_window:
        #     emotion_window.pop(0)
        #
        try:
            emotion_mode = mode(emotion_window)
            print("g")
        except:
            continue

        if emotion_text == 'angry':
            color = emotion_probability * np.asarray((255, 0, 0))
            Emotion_score_array.pop(0)
            Emotion_score_array.append(.22)
        elif emotion_text == 'sad':
            color = emotion_probability * np.asarray((0, 0, 255))
            Emotion_score_array.pop(0)
            Emotion_score_array.append(-0.4)
        elif emotion_text == 'happy':
            color = emotion_probability * np.asarray((255, 255, 0))
            Emotion_score_array.pop(0)
            Emotion_score_array.append(0.9)
        elif emotion_text == 'surprise':
            color = emotion_probability * np.asarray((0, 255, 255))
            Emotion_score_array.pop(0)
            Emotion_score_array.append(.61)
        elif emotion_text == 'disgust':
            color = emotion_probability * np.asarray((0, 255, 0))
            Emotion_score_array.pop(0)
            Emotion_score_array.append(.001)
        elif emotion_text == 'fear':
            color = emotion_probability * np.asarray((0, 255, 0))
            Emotion_score_array.pop(0)
            Emotion_score_array.append(.1)
        else:
            color = emotion_probability * np.asarray((0, 255, 0))
            print("i")

        color = color.astype(int)
        color = color.tolist()

        draw_bounding_box(face_coordinates, rgb_image, color)
        draw_text(face_coordinates, rgb_image, emotion_mode,
                  color, 0, -45, 1, 1)

    bgr_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR)
    #cv2.imshow('window_frame', bgr_image)
    # print(Emotion_score_array)
    i=i+1

    if i==100:
        Emotion_score=np.array(Emotion_score_array)
        a=np.mean(Emotion_score)+random.uniform(0.001,0.07)
        a=round(a,2)
        if a<=0.1:
            a=0.1
        if a>=0.99:
            a=0.99
        print(a)
        #timestamp = int(time.time())
        #fs(a,timestamp,u'actual')
        #appreciation
        if a<=0.65:
            ap=a+random.uniform(0.01,0.09)
        else:
            ap=a
        print(round(ap,2))
        # fs(round(ap,2),timestamp,u'appreciation')
        i=0

        def get_audio():
          display(HTML(AUDIO_HTML))
          data = eval_js("data")
          binary = b64decode(data.split(',')[1])

          process = (ffmpeg
            .input('pipe:0')
            .output('pipe:1', format='wav')
            .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
          )
          output, err = process.communicate(input=binary)

          riff_chunk_size = len(output) - 8
          # Break up the chunk size into four bytes, held in b.
          q = riff_chunk_size
          b = []
          for i in range(4):
              q, r = divmod(q, 256)
              b.append(r)

          # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
          riff = output[:4] + bytes(b) + output[8:]

          sr, audio = wav_read(io.BytesIO(riff))

          return audio, sr

        xyz = get_audio()
        print(xyz[0])
        fs = 44100
        seconds = 5
        os.chdir("/content/")
        y = (np.iinfo(np.int32).max * (xyz[0]/np.abs(xyz[0]).max())).astype(np.int32)  # You were passing the data as float but speech_recognition takes
        wavio.write("output.wav", xyz[0], fs ,sampwidth=2)

# RECOGNITION
        import speech_recognition as sr
        r=sr.Recognizer()
        recording = sr.AudioFile("/content/output.wav")
        with recording as source :
          audio1=r.record(source)

          try:
            state = r.recognize_google(audio1,language="en-GB")
          except:
            print("I can't get you. Please try again")
            break
          if state.lower()=="stop" :
            break

        print(state)
        feedback = state
        blob=TextBlob(feedback)
        #print(blob.sentiment)

        p,s = blob.sentiment
        e_ = p*s
        e = (abs(-0.5-(e_/2)))
        print(e)
        m = (ap+e)/2
        print(ap)
        print(e)
#SPOTIFY
        mood = m
        print("mood = ",mood)
        client_id = 'your-client-id-here'
        client_secret = 'your-client-secret-here'
        redirect_uri = 'your-redirect-url-here'
        scope = 'user-library-read user-top-read playlist-modify-public user-follow-read'
        username= 'your-username-here'
        token = utilx.prompt_for_user_token(username,scope,client_id=client_id,client_secret=client_secret,redirect_uri=redirect_uri)
        if token:

        #Step 1. Authenticating Spotipy

          def authenticate_spotify():
            print('...connecting to Spotify')
            sp = spotipy.Spotify(auth=token)
            return sp

        #Step 2. Creating a list of your favorite artists

          def aggregate_top_artists(sp):
            print('...getting your top artists')
            top_artists_name = []
            top_artists_uri = []

            ranges = ['short_term', 'medium_term', 'long_term']
            for r in ranges:
              top_artists_all_data = sp.current_user_top_artists(limit=50, time_range= r)
              top_artists_data = top_artists_all_data['items']
              for artist_data in top_artists_data:
                if artist_data["name"] not in top_artists_name:
                  top_artists_name.append(artist_data['name'])
                  top_artists_uri.append(artist_data['uri'])

            followed_artists_all_data = sp.current_user_followed_artists(limit=50)
            followed_artists_data = (followed_artists_all_data['artists'])
            for artist_data in followed_artists_data["items"]:
              if artist_data["name"] not in top_artists_name:
                top_artists_name.append(artist_data['name'])
                top_artists_uri.append(artist_data['uri'])
            return top_artists_uri


        #Step 3. For each of the artists, get a set of tracks for each artist

          def aggregate_top_tracks(sp, top_artists_uri):
            print("...getting top tracks")
            top_tracks_uri = []
            for artist in top_artists_uri:
              top_tracks_all_data = sp.artist_top_tracks(artist)
              top_tracks_data = top_tracks_all_data['tracks']
              for track_data in top_tracks_data:
                top_tracks_uri.append(track_data['uri'])
            return top_tracks_uri

        # Step 4. From top tracks, select tracks that are within a certain mood range

          def select_tracks(sp, top_tracks_uri):

            print("...selecting tracks")
            selected_tracks_uri = []

            def group(seq, size):
              return (seq[pos:pos + size] for pos in range(0, len(seq), size))

            random.shuffle(top_tracks_uri)
            for tracks in list(group(top_tracks_uri, 50)):
              tracks_all_data = sp.audio_features(tracks)
              for track_data in tracks_all_data:
                try:
                  if mood < 0.10:
                    if (0 <= track_data["valence"] <= (mood + 0.15)
                    and track_data["danceability"] <= (mood*8)
                    and track_data["energy"] <= (mood*10)):
                      selected_tracks_uri.append(track_data["uri"])
                  elif 0.10 <= mood < 0.25:
                    if ((mood - 0.075) <= track_data["valence"] <= (mood + 0.075)
                    and track_data["danceability"] <= (mood*4)
                    and track_data["energy"] <= (mood*5)):
                      selected_tracks_uri.append(track_data["uri"])
                  elif 0.25 <= mood < 0.50:
                    if ((mood - 0.05) <= track_data["valence"] <= (mood + 0.05)
                    and track_data["danceability"] <= (mood*1.75)
                    and track_data["energy"] <= (mood*1.75)):
                      selected_tracks_uri.append(track_data["uri"])
                  elif 0.50 <= mood < 0.75:
                    if ((mood - 0.075) <= track_data["valence"] <= (mood + 0.075)
                    and track_data["danceability"] >= (mood/2.5)
                    and track_data["energy"] >= (mood/2)):
                      selected_tracks_uri.append(track_data["uri"])
                  elif 0.75 <= mood < 0.90:
                    if ((mood - 0.075) <= track_data["valence"] <= (mood + 0.075)
                    and track_data["danceability"] >= (mood/2)
                    and track_data["energy"] >= (mood/1.75)):
                      selected_tracks_uri.append(track_data["uri"])
                  elif mood >= 0.90:
                    if ((mood - 0.15) <= track_data["valence"] <= 1
                    and track_data["danceability"] >= (mood/1.75)
                    and track_data["energy"] >= (mood/1.5)):
                      selected_tracks_uri.append(track_data["uri"])
                except TypeError as te:
                  continue

            return selected_tracks_uri

        # Step 5. From these tracks, create a playlist for user
          def generateOTP() :

            # Declare a digits variable
            # which stores all digits
            digits = "0123456789"
            OTP = ""

          # length of password can be chaged
          # by changing value in range
            for i in range(3) :
                OTP += digits[math.floor(random.random() * 10)]

            return OTP

          def create_playlist(sp, selected_tracks_uri):

            print("...creating playlist")
            user_all_data = sp.current_user()
            user_id = user_all_data["id"]
            opt = generateOTP()
            print("Name of new playlist:", opt+"_"+str(int(m*100)))
            playlist_all_data = sp.user_playlist_create(user_id, opt+"_"+str(int(m*100)) )#"Your mood= " + str(mood) + " @" + current_time + ", " + d4)
            playlist_id = playlist_all_data["id"]

            random.shuffle(selected_tracks_uri)
            sp.user_playlist_add_tracks(user_id, playlist_id, selected_tracks_uri[0:30])


          spotify_auth = authenticate_spotify()
          top_artists = aggregate_top_artists(spotify_auth)
          top_tracks = aggregate_top_tracks(spotify_auth, top_artists)
          selected_tracks = select_tracks(spotify_auth, top_tracks)
          create_playlist(spotify_auth, selected_tracks)

          filename = take_photo()
          print("-----------------------------------------------------------------------------------------------------------------------------")

<IPython.core.display.Javascript object>

photo.jpg


0.57
0.58


[  0   0   0 ...  13 -39  60]
the world seems so dull today
0.4270833333333333
0.5824045671425198
0.4270833333333333
mood =  0.5047439502379265
...connecting to Spotify
...getting your top artists
...getting top tracks
...selecting tracks
...creating playlist
Name of new playlist: 591_50


<IPython.core.display.Javascript object>

photo.jpg
-----------------------------------------------------------------------------------------------------------------------------
0.56
0.65


[    0     0     0 ... -1685 -1712 -1699]
I can't get you. Please try again
